In [1]:
import torch
import torch.nn as nn
import time
import os

## Load model

In [2]:
class FCN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FCN, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out) 
        return out

In [3]:
model = FCN(1,1,1)
model = torch.load('full_model.pth')
model

FCN(
  (l1): Linear(in_features=36, out_features=20, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=20, out_features=5, bias=True)
)

In [4]:
for weight in model.state_dict():
    print(weight)

l1.weight
l1.bias
l2.weight
l2.bias


In [5]:
for key, val in model.state_dict().items():
    print(key, val.shape)

l1.weight torch.Size([20, 36])
l1.bias torch.Size([20])
l2.weight torch.Size([5, 20])
l2.bias torch.Size([5])


## Print weights for HLS

In [6]:
l1_weight = model.state_dict()['l1.weight'].tolist()
l2_weight = model.state_dict()['l2.weight'].tolist()
l1_bias = model.state_dict()['l1.bias'].tolist()
l2_bias = model.state_dict()['l2.bias'].tolist()

In [7]:
print("const float l1_bias[HIDDEN_SIZE] = {", end='')
biases = ''
for b in l1_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l1_bias[HIDDEN_SIZE] = {0.07260576635599136, 0.2663082480430603, -0.35320067405700684, 0.26500728726387024, 0.08590759336948395, -0.3907392919063568, 0.13615305721759796, -0.5529122352600098, 0.6287691593170166, 0.10721774399280548, 0.02466747909784317, 0.8518648743629456, -0.04959215596318245, -0.04843122512102127, -0.6003413200378418, -0.1684667021036148, -0.009274917654693127, 0.6424950361251831, -0.16273681819438934, 0.23099926114082336};


In [8]:
print("const float l2_bias[NUM_CLASSES] = {", end='')
biases = ''
for b in l2_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l2_bias[NUM_CLASSES] = {0.9476150870323181, -0.06746087223291397, -0.4606860876083374, -0.4210861027240753, 0.15140864253044128};


In [9]:
print("const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {", end='')
weights = ''
for row in l1_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {{0.017793796956539154, 0.0067334482446312904, 0.1723751425743103, -0.02333994023501873, -0.07630836963653564, -0.09063716232776642, 0.09040980786085129, -0.18284820020198822, 0.08473024517297745, 0.08516852557659149, -0.0866696760058403, -0.14805960655212402, -0.16540396213531494, -0.02581767924129963, -0.11177141219377518, -0.07134535908699036, -0.13314352929592133, -0.15574730932712555, -0.16199088096618652, -0.0019097545882686973, 0.12946054339408875, -0.1012590229511261, -0.07195274531841278, 0.05875340849161148, -0.042263198643922806, 0.09179727733135223, 0.11246266216039658, -0.11463310569524765, 0.08561989665031433, -0.012116599828004837, 0.05730132386088371, -0.03909002244472504, 0.10171392560005188, -0.14406642317771912, 0.026497645303606987, 0.12936124205589294},
{-0.014365200884640217, 0.3485833406448364, 1.3608559370040894, -0.15878449380397797, 0.7325801849365234, 0.3468405604362488, -0.5683023929595947, 0.0531378239393234

In [10]:
print("const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {", end='')
weights = ''
for row in l2_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {{-0.09548430144786835, -0.03395885229110718, -0.09811938554048538, -0.04564622789621353, 0.028937123715877533, -0.16248704493045807, 0.10498672723770142, -0.16764508187770844, 0.28133320808410645, 0.140278160572052, -0.10908842086791992, 0.3037027418613434, 0.04044416919350624, 0.06996949017047882, -0.198881134390831, -0.2094779759645462, -0.014100474305450916, 0.6162449717521667, 0.0985562652349472, 0.423587441444397},
{0.12505847215652466, -0.17951709032058716, 0.09106337279081345, 0.1357848346233368, 0.20477169752120972, 0.12727369368076324, 0.19728487730026245, 0.2695009708404541, -0.7718627452850342, 0.14876717329025269, -0.32329100370407104, -0.2513580620288849, 0.16203387081623077, -0.22017818689346313, 0.06506700813770294, 0.1581232100725174, -0.05689410865306854, -0.20234188437461853, 0.027573073282837868, -0.16650605201721191},
{0.11324868351221085, -0.07964815199375153, -0.017609208822250366, -0.26008614897727966, -0.184993

## Get test data for software-hardware comparison

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing

In [12]:
DATA_DIR = 'data'
RAW = DATA_DIR + '/raw'
NUM_FEATURES = 36

In [13]:
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
X_test = test_df.drop('action', axis='columns')
y_test = test_df['action']

In [14]:
data = [torch.Tensor(X_test.iloc[x].astype(float)) for x in range(5)]

In [15]:
data

[tensor([ 3.1000e-01, -1.0800e+00, -7.1600e-02,  2.5055e-01,  2.2250e-01,
          1.8464e-01,  1.1500e+00, -7.5000e-01,  3.3620e-01,  4.5541e-01,
          4.9750e-01,  3.6738e-01,  8.9000e-01, -1.5800e+00, -7.4200e-02,
          6.3124e-01,  9.4500e-01,  5.4274e-01,  1.6082e+02, -2.5134e+02,
         -7.1260e+00,  8.0422e+01,  4.4533e+01,  5.0580e+01,  2.6117e+02,
         -2.8273e+02, -2.4086e+00,  1.2021e+02,  5.4765e+01,  7.7564e+01,
          2.7862e+02, -2.6073e+02, -9.0498e+00,  9.0063e+01,  4.3487e+01,
          5.4934e+01]),
 tensor([ 1.4800e+00, -6.9000e-01, -1.7820e-01,  4.0969e-01,  2.2000e-01,
          2.5765e-01,  1.8300e+00, -1.1100e+00,  1.2720e-01,  6.1517e-01,
          4.8250e-01,  4.5133e-01,  1.2800e+00, -1.2700e+00, -1.9820e-01,
          4.3570e-01,  4.1750e-01,  3.1715e-01,  1.6864e+02, -3.1762e+02,
         -3.6890e+01,  1.1007e+02,  1.2509e+02,  8.5393e+01,  2.3456e+02,
         -2.8976e+02, -3.4642e+01,  9.6951e+01,  6.0792e+01,  6.3434e+01,
          1.65

In [16]:
with torch.no_grad():
    for d in data:
        start_time = time.time()
        output = model(d)
        action = np.argmax(output)
#         print(f"Inference time: {time.time() - start_time}")    
        print(action)

tensor(3)
tensor(4)
tensor(1)
tensor(1)
tensor(4)


In [17]:
for x in range(5):
    print(y_test[x])

3
4
1
1
3


## Compare software-hardware running time and accuracy

In [18]:
# 1 sample

with torch.no_grad():
    start_time = time.time()

    data = torch.Tensor(X_test.iloc[0].astype(float))
    output = model(data)
    action = np.argmax(output)

    end_time = time.time()
    print(f"Prediction: {action}; Ground truth value: {y_test[0]}")
    print(f"Inference time: {((end_time - start_time) * 1000):.4f} ms") 

Prediction: 3; Ground truth value: 3
Inference time: 0.5820 ms


In [19]:
# whole test set (or dev + test set used in training)

with torch.no_grad():
    start_time = time.time()
    
    num_correct = 0
    num_samples = 0

    for i in range(len(X_test[:])):
        data = torch.Tensor(X_test.iloc[i].astype(float))
        output = model(data)
        action = np.argmax(output)
        num_samples += 1
        num_correct += (action == y_test[i])

    end_time = time.time()
    print(f"Accuracy: {(100.0 * num_correct / num_samples):.2f}%")
    print(f"Inference time: {(end_time - start_time):.4f} s")    

Accuracy: 91.96%
Inference time: 0.0685 s


In [20]:
len(X_test)

112